# Function Calling with GPT Models

This notebook explores the integration of GPT models with external functions. We'll demonstrate how to call functions within the GPT context, convert Python functions to a JSON-compatible format, and execute functions retrieved from GPT outputs.

### Content Overview:

**I. [Understanding Function Calls in GPT](#section_one)** </br>
>- Explore how GPT models can interpret and utilize JSON-formatted functions.
>- Learn about the necessary components of a JSON function for GPT compatibility.

**II. [Python to JSON Function Conversion](#section_two)** </br>
>- Step-by-step guide to transforming a Python function into a JSON format.
>- Methods for passing JSON functions to a GPT model for processing.

**III. [Executing Functions from GPT Outputs](#section_three)** </br>
>- Detailed example on how to retrieve a function from GPT output and execute it in Python.
>- A good practice for handling and validating the execution of functions from GPT.

**Extra read:**
>- https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb by Jeremy Howard - [YouTube Link](https://www.youtube.com/watch?v=jkrNMKz9pWU)

**NOTE:**
- This code requires pydantic==2.5.1 for successful exection.

### OpenAI chat completion arguments:

Below is a sample code snippet that demonstrates how to use the OpenAI Chat Completion API with function calling:

```
response = openai.ChatCompletion.create(
    engine=gpt_model, # Name of the gpt model : str
    messages=messages, # Input messages: List[Dict]
    functions=search_hotels_function_json, # List of functions in JSON format: List
    function_call="auto", # 'auto': GPT model decides when to use functions. None: GPT model will not use the functions. Dict {"name": "function_name"}: GPT model will always use the defined functions.
    temperature=temperature # 0 to 1: float
)
```

In [86]:
# import libraries
from pydantic import create_model
import inspect, json
from inspect import Parameter
import openai
import os
from dotenv import load_dotenv

load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [87]:
# define the necessary configs
gpt_model = "gpt-35-turbo-16k"
temperature = 0

<a id=section_one></a>
**Understanding Function Calls in GPT**

In [89]:
messages= [
    {"role": "user", "content": "What is the result of 3+5?"}
]

# Pass th function toGPT model
response = openai.ChatCompletion.create(
    engine=gpt_model,
    messages=messages,
    # functions=sum,
    # function_call="auto",
    temperature=temperature
)

In [91]:
print(response.choices[0] "\n")

{
  "finish_reason": "stop",
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "The result of 3+5 is 8."
  },
  "content_filter_results": {
    "hate": {
      "filtered": false,
      "severity": "safe"
    },
    "self_harm": {
      "filtered": false,
      "severity": "safe"
    },
    "sexual": {
      "filtered": false,
      "severity": "safe"
    },
    "violence": {
      "filtered": false,
      "severity": "safe"
    }
  }
} 



In [92]:
sum = [  
    {'name': 'sum',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sum`',
  'type': 'object'}}
]

messages= [
    {"role": "user", "content": "What is the result of 3+5?"}
]

# Pass th function toGPT model
response = openai.ChatCompletion.create(
    engine=gpt_model,
    messages=messages,
    functions=sum,
    function_call="auto",
    temperature=temperature
)

In [94]:
response

<OpenAIObject chat.completion id=chatcmpl-8WveEzI0uoPJflKYPK11vctJ9gOlj at 0x1eecc281f10> JSON: {
  "id": "chatcmpl-8WveEzI0uoPJflKYPK11vctJ9gOlj",
  "object": "chat.completion",
  "created": 1702858786,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "sum",
          "arguments": "{\n  \"a\": 3,\n  \"b\": 5\n}"
        }
      },
      "content_filter_results": {}
    }


In [95]:
print("Full response:")
print(response.choices[0].message, "\n")
print("\nFunction call:")
print(response.choices[0].message.function_call, "\n")
print("\nFunction call arguments:")
print(response.choices[0].message.function_call.arguments, "\n")

Full response:
{
  "role": "assistant",
  "function_call": {
    "name": "sum",
    "arguments": "{\n  \"a\": 3,\n  \"b\": 5\n}"
  }
} 


Function call:
{
  "name": "sum",
  "arguments": "{\n  \"a\": 3,\n  \"b\": 5\n}"
} 


Function call arguments:
{
  "a": 3,
  "b": 5
} 



<a id=section_two></a>
**Python to JSON Function Conversion**

In [96]:
def sum(a:int, b:int=1):
    "Adds a + b"
    return a + b

def subtract(a:int, b:int=1):
    "Subtracts a - b"
    return a - b

def multiply(a:int, b:int=1):
    "Multiplies a * b"
    return a * b

def jsonschema(f):
        """
        Generate a JSON schema for the input parameters of the given function.

        Parameters:
            f (FunctionType): The function for which to generate the JSON schema.

        Returns:
            Dict: A dictionary containing the function name, description, and parameters schema.
        """
        kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
              for n, o in inspect.signature(f).parameters.items()}
        s = create_model(f'Input for `{f.__name__}`', **kw).schema()
        return dict(name=f.__name__, description=f.__doc__, parameters=s)

def execute_json_function(response):
        """
        Execute a function based on the response from an OpenAI ChatCompletion API call.

        Parameters:
            response: The response object from the OpenAI ChatCompletion API call.

        Returns:
            List: The result of the executed function.
        """
        func_name = response.choices[0].message.function_call.name
        func_args = json.loads(
            response.choices[0].message.function_call.arguments)
        # Call the function with the given arguments
        if func_name == 'sum':
            result = sum(**func_args)
        elif func_name == 'subtract':
            result = subtract(**func_args)
        elif func_name == 'multiply':
            result = multiply(**func_args)
        else:
            raise ValueError(f"Function '{func_name}' not found.")
        return result

print(type(jsonschema(sum)), "\n")
jsonschema(sum)

<class 'dict'> 



C:\Users\farza\AppData\Local\Temp\ipykernel_14316\3158027094.py:25: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'sum',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sum`',
  'type': 'object'}}

When functions are provided, by default the function_call will be set to "auto" and the model will decide whether or not a function should be called. 

In [99]:
llm_system_role = "Answer the user's question."
prompt = "What is 6*3?"

response = openai.ChatCompletion.create(
            engine=gpt_model,
            messages=[
                {"role": "system", "content": llm_system_role},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            functions=[jsonschema(sum), jsonschema(subtract), jsonschema(multiply)],
            function_call="auto",            
        )

C:\Users\farza\AppData\Local\Temp\ipykernel_14316\3158027094.py:25: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [100]:
response

<OpenAIObject chat.completion id=chatcmpl-8Wvhlp9NyN8PRY04mhopzradCgW83 at 0x1eecc282c90> JSON: {
  "id": "chatcmpl-8Wvhlp9NyN8PRY04mhopzradCgW83",
  "object": "chat.completion",
  "created": 1702859005,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "multiply",
          "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
        }
      },
      "content_filter_results": {}
 

In [101]:
print("Full response:")
print(response.choices[0].message, "\n")
print("\nFunction call:")
print(response.choices[0].message.function_call, "\n")
print("\nFunction call arguments:")
print(response.choices[0].message.function_call.arguments, "\n")

Full response:
{
  "role": "assistant",
  "function_call": {
    "name": "multiply",
    "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
  }
} 


Function call:
{
  "name": "multiply",
  "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
} 


Function call arguments:
{
  "a": 6,
  "b": 3
} 



In [102]:
response.choices[0].message.function_call

<OpenAIObject at 0x1eecc2830b0> JSON: {
  "name": "multiply",
  "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
}

In [103]:
execute_json_function(response)

18